<a href="https://colab.research.google.com/github/kjoachim/a301_extras/blob/kj/notebooks/week10/read_landsat_ee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# An Intro to the Earth Engine Python API

Adapted from the Google earth engine example page for ATSC 301
-- under construction

Short demo to show some of the functionality of the
[Google Earth Engine Python API](https://developers.google.com/earth-engine/guides/python_install) is presented.

Specificaly -- we:

- search for landsat images  within a date range containing
  a point (UBC Vancouver)
- filter the images for cloud cover
- clip to a region of interest
- mask for cloud and water
- extract band 5 and scale it
- display band 5 on a map
- write it out as a tiff file
- mount a google drive and read it back in using rioxarray
- plot it with rioxarray


In [26]:
# uncomment pip to install geemap
!pip install geemap

In [27]:
import ee
import geemap.core as geemap
import geemap as gem

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-joachkay')

## Search for cloud-free images

In [28]:
ubc_lon = -123.2460 #degrees West
ubc_lat = 49.2606  # degrees North

Find scenes in July and August between 2000 and 2002 that contain UBC

In [29]:
# Load Landsat 7 data, filter by date, month, and bounds.
collection = (
    ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
    # Two years of data
    .filterDate('2004-01-01', '2006-01-01')
    # Only July-Aug observations
    .filter(ee.Filter.calendarRange(7, 8, 'month'))
    # Intersecting ROI
    .filterBounds(ee.Geometry.Point(ubc_lon, ubc_lat))
)

In [30]:
cloud_cover_values = collection.aggregate_array('CLOUD_COVER').getInfo()
print(cloud_cover_values)

[23, 1, 92, 74, 3, 9, 66, 31, 28, 21, 79, 10, 0, 66]


## Filter on cloud cover

In [31]:
# prompt: filter on cloud fraction less than 10

# Filter the collection to select scenes with less than 10% cloud cover
collection = collection.filter(ee.Filter.lt('CLOUD_COVER', 10))

# Print the number of scenes in the filtered collection
print('Number of scenes with less than 10% cloud cover:', collection.size().getInfo())

# ... (rest of your code)


Number of scenes with less than 10% cloud cover: 4


## Clip to a region

In [32]:
 # prompt: clip collection to a roi of 10 km around (ubc_lon, ubc_lat)

# Define the region of interest (ROI)
roi = ee.Geometry.Point(ubc_lon, ubc_lat).buffer(10000) # 10 km buffer

# Clip the image collection to the ROI
clipped_collection = collection.map(lambda image: image.clip(roi))

# Print the number of scenes in the clipped collection
print('Number of scenes in the clipped collection:', clipped_collection.size().getInfo())

Number of scenes in the clipped collection: 4


## Mask on clouds and water

In [33]:
# prompt: mask the image for cloud and water

# Function to mask clouds and water
def maskCloudsAndWater(image):
    qa = image.select('QA_PIXEL')
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloud = qa.bitwiseAnd(1 << 3).eq(0)
    cloudShadows = qa.bitwiseAnd(1 << 5).eq(0)
    # Water bit
    water = qa.bitwiseAnd(1 << 2).eq(0)  # Assuming bit 2 represents water
    mask = cloud.And(cloudShadows).And(water)
    return image.updateMask(mask)

# Apply the cloud and water masking function to the image collection
masked_collection = clipped_collection.map(maskCloudsAndWater)


## Select and scale band 5

In [34]:
image2 = ee.Image(masked_collection.toList(masked_collection.size()).get(1))
band_sr5 = image2.select('SR_B5')

In [35]:
scale_factor = 0.0000275
add_factor = -0.2
band_sr5= band_sr5.multiply(scale_factor).add(add_factor)

## Display on a map

In [36]:
Map = gem.Map()
Map.centerObject(ee.Geometry.Point([ubc_lon, ubc_lat]),  12)

# Add the image to the map.
vis_params = {'min': 0, 'max': 0.3}
Map.addLayer(band_sr5, vis_params, 'Landsat 7 SR_B5')

# Display the map.
Map


Map(center=[49.2606, -123.24600000000001], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

## Export to a tiff file

Note that the file won't be overwritten, if you want to replace
it you need to use the pathlib module to check if it exists and
then unlink

### Delete the file if it exists

In [37]:
filename = "band_sr5_masked.tif"
folder = "earth_engine_downloads"
from pathlib import Path
the_file = Path(f"/content/drive/MyDrive/{folder}").glob(f"**/{filename}")
the_file = list(the_file)
if the_file:
  the_file[0].unlink()

### Write out the tiff

Strip the `.tif` off of the file name

In [38]:
filestem = Path(filename).stem
filestem

'band_sr5_masked'

### Option 1: save it to your google drive

In [39]:
# prompt: save band_sr5 as a downloadable tiff file

# Export the image as a GeoTIFF to your Google Drive.
task = ee.batch.Export.image.toDrive(**{
    'image': band_sr5,
    'description': 'band_sr5_export',
    'folder': folder,
    'fileNamePrefix': filestem,
    'region': roi,
    'scale': 30,
    'maxPixels': 1e13
})
task.start()

# Wait for the export task to complete (optional)
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)
print('Task completed!')





Polling for task (id: 54LE7YSI5WBRITBSNWLHZD7L).
Polling for task (id: 54LE7YSI5WBRITBSNWLHZD7L).
Polling for task (id: 54LE7YSI5WBRITBSNWLHZD7L).
Polling for task (id: 54LE7YSI5WBRITBSNWLHZD7L).
Polling for task (id: 54LE7YSI5WBRITBSNWLHZD7L).
Task completed!


### Option 2: provide a url to download to local disk

Copy and paste the url in your browser and download the zip file containing the geotif

In [40]:
# prompt: band_sr5  as geotiff using  ee.image.getdownloadurl

# Get the download URL for the image.
download_url = band_sr5.getDownloadURL({
    'name': filestem,
    'scale': 30,
    'region': roi,
    'format': 'GeoTIFF'
})

download_url

'https://earthengine.googleapis.com/v1/projects/ee-joachkay/thumbnails/14749bdaad38e94ef7e6e3f7077909bf-6ecae5251902a91e8fe6d19e4bdd94f1:getPixels'

## Mount the google drive and read the image


In [ ]:
# prompt: hello world function

def hello_world():
  """Prints 'Hello, world!' to the console."""
  print("Hello, world!")

# Call the function to execute it
hello_world()


In [45]:
#this only works on collab
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!ls -l /content/drive/MyDrive/earth_engine_downloads

## Read and plot the image with rioxarray

Uncomment the next line to install rioxarray

In [ ]:
#!pip install rioxarray

In [ ]:
import rioxarray
band5 = rioxarray.open_rasterio('/content/drive/MyDrive/earth_engine_downloads/band_sr5_masked.tif',mask_and_scale=True)
band5 = band5.squeeze()
band5.plot.imshow(vmin=0,vmax=0.3);

In [ ]:
band5.plot.hist();

## Doing batch downloads

Haven't tried this, but [geedim](https://github.com/leftfield-geospatial/geedim) looks promissing for batch downloads